In [9]:
import FinanceDataReader as fdr
import pandas as pd
from datetime import datetime

# 오늘 날짜를 'yy-mm-dd' 형식으로 가져오기
end_date = datetime.today().strftime('%y-%m-%d')

# 파일 경로
file_path = 'invest_universe.csv'

# 데이터 로드
tickers_data = pd.read_csv(file_path, encoding='cp949')  # 한글 인코딩 처리

# '종목 코드' 컬럼을 문자열로 변환하고 6자리로 맞추기
tickers_data['종목 코드'] = tickers_data['종목 코드'].astype(str).str.zfill(6)

# Ticker 리스트 추출
tickers = tickers_data['종목 코드']

# 함수 정의: Tickers를 기준으로 데이터 가져오기
def fetch_close_prices(tickers, start_date='2000-01-01', end_date=end_date):
    """
    여러 종목의 종가 데이터를 하나의 DataFrame으로 저장
    :param tickers: 종목 티커 리스트
    :param start_date: 데이터 시작 날짜
    :param end_date: 데이터 종료 날짜
    :return: 하나의 DataFrame (날짜 기준 정렬, 결측값 포함)
    """
    all_close_prices = pd.DataFrame()
    


    for ticker in tickers:
        try:
            # 티커 데이터 읽기
            df = fdr.DataReader(ticker, start=start_date, end=end_date)
            # 인덱스를 yyyy-mm-dd 형식으로 변환
            df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')
            # 종가 데이터만 추출
            close_prices = df['Close']
            # 컬럼 이름 변경 (티커명)
            close_prices.name = ticker
            # 데이터프레임 병합 (outer join)
            all_close_prices = pd.concat([all_close_prices, close_prices], axis=1)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    
    return all_close_prices

# 함수 실행
close_prices_df = fetch_close_prices(tickers)

# 인덱스를 datetime으로 변환
close_prices_df.index = pd.to_datetime(close_prices_df.index)

# 결과 저장
close_prices_df.to_csv('db.csv')

# 결과 확인
close_prices_df.head()


,069500,122630,278530,233740,102780,229200,305540,139260,091180,462010,...,144600,261220,400570,352560,138230,271060,160580,130680,225800,352540
2002-10-14,5145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-15,5272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-16,5294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-17,5369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-18,5599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
close_prices_df.tail()

,069500,122630,278530,233740,102780,229200,305540,139260,091180,462010,...,144600,261220,400570,352560,138230,271060,160580,130680,225800,352540
2024-11-19,32945,14835.0,11555.0,7395.0,8240.0,11740.0,16950.0,26885.0,18210.0,4640.0,...,5665.0,14350.0,10215.0,12620.0,15470.0,8465.0,11075.0,4190.0,14490.0,10930.0
2024-11-20,33010,14875.0,11575.0,7260.0,8255.0,11645.0,17105.0,26880.0,18445.0,4665.0,...,5600.0,14340.0,10075.0,12690.0,15470.0,8455.0,11100.0,4195.0,14490.0,10930.0
2024-11-21,33050,14930.0,11585.0,7275.0,8315.0,11665.0,17435.0,27005.0,18285.0,4820.0,...,5640.0,14295.0,10110.0,12625.0,15510.0,8485.0,11175.0,4185.0,14615.0,10880.0
2024-11-22,33345,15170.0,11685.0,6980.0,8360.0,11400.0,17610.0,27495.0,18325.0,4890.0,...,5595.0,14520.0,10405.0,12730.0,15580.0,8415.0,11130.0,4225.0,14690.0,10940.0
2024-11-25,33735,15530.0,11825.0,7565.0,8495.0,11905.0,18115.0,27970.0,18185.0,5070.0,...,5545.0,14630.0,10250.0,12880.0,15560.0,8390.0,11115.0,4260.0,14680.0,10935.0


In [11]:
close_prices_df.to_csv('db.csv')

In [24]:
import FinanceDataReader as fdr
import pandas as pd
import sqlite3

# 파일 경로
file_path = 'invest_universe.csv'

# SQLite3 데이터베이스 경로
db_path = 'financial_data.db'

# 데이터 로드
tickers_data = pd.read_csv(file_path, encoding='cp949')  # 한글 인코딩 처리

# '종목 코드' 컬럼을 문자열로 변환하고 6자리로 맞추기
tickers_data['종목 코드'] = tickers_data['종목 코드'].apply(lambda x: f"{int(x):06}")

# Ticker 리스트 추출
tickers = tickers_data['종목 코드']

# 함수 정의: Tickers를 기준으로 데이터 가져오기
def fetch_close_prices(tickers, start_date='2000-01-01', end_date=end_date):
    """
    여러 종목의 종가 데이터를 하나의 DataFrame으로 저장
    :param tickers: 종목 티커 리스트
    :param start_date: 데이터 시작 날짜
    :param end_date: 데이터 종료 날짜
    :return: 하나의 DataFrame (날짜 기준 정렬, 결측값 포함)
    """
    all_close_prices = pd.DataFrame()
    
    for ticker in tickers:
        try:
            # 티커 데이터 읽기
            df = fdr.DataReader(ticker, start=start_date, end=end_date)
            # 인덱스를 yyyy-mm-dd 형식으로 변환
            df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')
            # 종가 데이터만 추출
            close_prices = df['Close']
            # 컬럼 이름 변경 (티커명)
            close_prices.name = ticker
            # 데이터프레임 병합 (outer join)
            all_close_prices = pd.concat([all_close_prices, close_prices], axis=1)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    
    return all_close_prices

# 함수 실행
close_prices_df = fetch_close_prices(tickers)

# 인덱스를 datetime으로 변환
close_prices_df.index = pd.to_datetime(close_prices_df.index)

# SQLite 데이터베이스 연결
conn = sqlite3.connect(db_path)

# 데이터 저장
try:
    close_prices_df.to_sql('db', conn, if_exists='replace', index_label='Date')
    print(f"Data successfully saved to SQLite database at {db_path}")
except Exception as e:
    print(f"Error saving to database: {e}")
finally:
    conn.close()

# 결과 확인
print(close_prices_df.head())


Data successfully saved to SQLite database at financial_data.db
            069500  122630  278530  233740  102780  229200  305540  139260  \
2002-10-14    5145     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2002-10-15    5272     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2002-10-16    5294     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2002-10-17    5369     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2002-10-18    5599     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

            091180  462010  ...  144600  261220  400570  352560  138230  \
2002-10-14     NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN   
2002-10-15     NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN   
2002-10-16     NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN   
2002-10-17     NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN   
2002-10-18     NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN   

            2710

In [25]:
close_prices_df

,069500,122630,278530,233740,102780,229200,305540,139260,091180,462010,...,144600,261220,400570,352560,138230,271060,160580,130680,225800,352540
2002-10-14,5145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-15,5272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-16,5294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-17,5369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-18,5599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-19,32945,14835.0,11555.0,7395.0,8240.0,11740.0,16950.0,26885.0,18210.0,4640.0,...,5665.0,14350.0,10215.0,12620.0,15470.0,8465.0,11075.0,4190.0,14490.0,10930.0
2024-11-20,33010,14875.0,11575.0,7260.0,8255.0,11645.0,17105.0,26880.0,18445.0,4665.0,...,5600.0,14340.0,10075.0,12690.0,15470.0,8455.0,11100.0,4195.0,14490.0,10930.0
2024-11-21,33050,14930.0,11585.0,7275.0,8315.0,11665.0,17435.0,27005.0,18285.0,4820.0,...,5640.0,14295.0,10110.0,12625.0,15510.0,8485.0,11175.0,4185.0,14615.0,10880.0
2024-11-22,33345,15170.0,11685.0,6980.0,8360.0,11400.0,17610.0,27495.0,18325.0,4890.0,...,5595.0,14520.0,10405.0,12730.0,15580.0,8415.0,11130.0,4225.0,14690.0,10940.0


In [29]:
import sqlite3
import pandas as pd

def fetch_data_from_db(tickers, db_path='financial_data.db'):
    """
    SQLite3 데이터베이스에서 종목 리스트에 해당하는 시계열 데이터를 조회하여 DataFrame으로 반환
    :param tickers: 조회할 종목 리스트
    :param db_path: SQLite3 데이터베이스 경로
    :return: 선택한 종목들의 시계열 데이터를 포함한 DataFrame
    """
    try:
        # 데이터베이스 연결
        conn = sqlite3.connect(db_path)

        # 컬럼 이름을 대괄호로 감싸서 쿼리 작성
        columns = ', '.join([f'"{ticker}"' for ticker in tickers])
        query = f"""
        SELECT Date, {columns}
        FROM db
        """

        # 데이터베이스에서 데이터 읽기
        df = pd.read_sql_query(query, conn, index_col='Date')
        conn.close()

        # 인덱스를 datetime으로 변환
        df.index = pd.to_datetime(df.index)
        return df

    except Exception as e:
        print(f"Error fetching data from database: {e}")
        return pd.DataFrame()


# 사용 예시
selected_tickers = ['069500', '122630', '278530', '233740', '102780']
result_df = fetch_data_from_db(selected_tickers)

# 결과 출력
print(result_df.head())


            069500  122630  278530  233740  102780
Date                                              
2002-10-14    5145     NaN     NaN     NaN     NaN
2002-10-15    5272     NaN     NaN     NaN     NaN
2002-10-16    5294     NaN     NaN     NaN     NaN
2002-10-17    5369     NaN     NaN     NaN     NaN
2002-10-18    5599     NaN     NaN     NaN     NaN


In [30]:
result_df

,069500,122630,278530,233740,102780
Date,,,,,
2002-10-14,5145,NaN,NaN,NaN,NaN
2002-10-15,5272,NaN,NaN,NaN,NaN
2002-10-16,5294,NaN,NaN,NaN,NaN
2002-10-17,5369,NaN,NaN,NaN,NaN
2002-10-18,5599,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2024-11-19,32945,14835.0,11555.0,7395.0,8240.0
2024-11-20,33010,14875.0,11575.0,7260.0,8255.0
2024-11-21,33050,14930.0,11585.0,7275.0,8315.0
